# Setup AWS EventBridge To Trigger a Pipeline Execution with S3

In [ ]:
import os
import sagemaker
import logging
import boto3
import sagemaker
import pandas as pd
import json
from botocore.exceptions import ClientError

sess   = sagemaker.Session()
bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)
account_id = boto3.client('sts').get_caller_identity().get('Account')

## Steps
1. Create S3 Buckets
2. Enable CloudTrail Logging
3. Get StepFunctions Pipeline
4. Create EventBridge Rule
5. Test Trigger

# Create S3 Data Upload Bucket (watched) & S3 Bucket for CloudTrail Logs

In [ ]:
watched_bucket = 'dsoaws-test-upload-{}'.format(account_id)
print(watched_bucket)

In [ ]:
!aws s3 mb s3://$watched_bucket

In [ ]:
!aws s3 ls $watched_bucket

In [ ]:
cloudtrail_bucket = 'cloudtrail-dsoaws-{}'.format(account_id)
print(cloudtrail_bucket)

In [ ]:
!aws s3 mb s3://$cloudtrail_bucket

In [ ]:
!aws s3 ls $cloudtrail_bucket

# Attach an S3 Policy to the Cloud Trail ^^ Logging Bucket ^^ Above

In [ ]:
policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AWSCloudTrailAclCheck20150319",
            "Effect": "Allow",
            "Principal": {
                "Service": "cloudtrail.amazonaws.com"
            },
            "Action": "s3:GetBucketAcl",
            "Resource": "arn:aws:s3:::{}".format(cloudtrail_bucket)
        },
        {
            "Sid": "AWSCloudTrailWrite20150319",
            "Effect": "Allow",
            "Principal": {
                "Service": "cloudtrail.amazonaws.com"
            },
            "Action": "s3:PutObject",
            "Resource": "arn:aws:s3:::{}/AWSLogs/{}/*".format(cloudtrail_bucket, account_id),
            "Condition": {
                "StringEquals": {
                    "s3:x-amz-acl": "bucket-owner-full-control"
                }
            }
        },
        {
            "Sid": "AWSCloudTrailHTTPSOnly20180329",
            "Effect": "Deny",
            "Principal": {
                "Service": "cloudtrail.amazonaws.com"
            },
            "Action": "s3:*",
            "Resource": [
                "arn:aws:s3:::{}/AWSLogs/{}/*".format(cloudtrail_bucket, account_id),
                "arn:aws:s3:::{}".format(cloudtrail_bucket)
            ],
            "Condition": {
                "Bool": {
                    "aws:SecureTransport": "false"
                }
            }
        }
    ]
}

print(policy)

In [ ]:
policy_json = json.dumps(policy)

In [ ]:
with open("policy.json", "w") as outfile: 
    json.dump(policy, outfile)

In [ ]:
!cat policy.json

In [ ]:
!aws s3api put-bucket-policy --bucket $cloudtrail_bucket --policy file://policy.json

# Get Default Cloud Trail

In [ ]:
cloudtrail = boto3.client('cloudtrail')
s3 = boto3.client('s3')

In [ ]:
trails = cloudtrail.describe_trails()

In [ ]:
print(trails)

In [ ]:
if len(trails['trailList']) > 0:
    trail_name = trails['trailList'][0]['Name']
    trail_arn = trails['trailList'][0]['TrailARN']
    print(trail_name)
    print(trail_arn)
else:
    print("No existing Cloud Trail.")
    cloudtrail_bucket = 'cloudtrail-dsoaws-{}'.format(account_id)
    s3.create_bucket(ACL='private', Bucket=cloudtrail_bucket)
    t = cloudtrail.create_trail(Name='dsoaws', S3BucketName=cloudtrail_bucket)
    trail_name = t['Name']
    trail_arn = t['TrailARN']
    cloudtrail.start_logging(Name=trail_arn)
    print("Cloud Trail created. Started logging.")

## Get Default EventBridge EventBus

In [ ]:
events = boto3.client('events')

In [ ]:
response = events.describe_event_bus(Name='default')
eventbus_arn = response['Arn']
print(eventbus_arn)

## Create Data Event Logging on CloudTrail for our S3 bucket

In [ ]:
!aws cloudtrail get-event-selectors --trail-name $trail_name


In [ ]:
watched_bucket_arn = "arn:aws:s3:::{}/".format(watched_bucket)
print(watched_bucket_arn)

In [ ]:
command = '\'[{ "ReadWriteType": "WriteOnly", "IncludeManagementEvents":true, "DataResources": [{ "Type": "AWS::S3::Object", "Values": ["' + watched_bucket_arn + '"] }] }]\''


In [ ]:
print(command)

In [ ]:
!aws cloudtrail put-event-selectors --trail-name $trail_name --event-selectors $command

## Create Custom EventBridge Rule

In [ ]:
pattern = {
  "source": [
    "aws.s3"
  ],
  "detail-type": [
    "AWS API Call via CloudTrail"
  ],
  "detail": {
    "eventSource": [
      "s3.amazonaws.com"
    ],
    "eventName": [
      "PutObject"
    ],
    "requestParameters": {
      "bucketName": [
        "{}".format(watched_bucket)
      ]
    }
  }
}

print(pattern)

In [ ]:
pattern_json = json.dumps(pattern)

In [ ]:
response = events.put_rule(
    Name='S3-Trigger',
    EventPattern=pattern_json,
    State='ENABLED',
    Description='Triggers an event on S3 PUT',
    EventBusName='default'
)
print(response)

In [ ]:
rule_arn = response['RuleArn']
print(rule_arn)

# Add Target

## Create IAM Role

In [ ]:
iam = boto3.client('iam')

In [ ]:
iam_role_name_eventbridge = 'DSOAWS_EventBridge_Invoke_StepFunctions'

### Create AssumeRolePolicyDocument

In [ ]:
assume_role_policy_doc = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "events.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
}

In [ ]:
try:
    iam_role_eventbridge = iam.create_role(
        RoleName=iam_role_name_eventbridge,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy_doc),
        Description='DSOAWS EventBridge Role'
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Role already exists")
    else:
        print("Unexpected error: %s" % e)

### Get the Role ARN

In [ ]:
role_eventbridge = iam.get_role(RoleName=iam_role_name_eventbridge)
iam_role_eventbridge_arn = role_eventbridge['Role']['Arn']
print(iam_role_eventbridge_arn)

# Get the StepFunctions ARN and Name

In [ ]:
%store -r stepfunction_arn

In [ ]:
print(stepfunction_arn)

In [ ]:
%store -r stepfunction_name

In [ ]:
print(stepfunction_name)

# Define Eventbridge Policy

In [ ]:
eventbridge_sfn_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VisualEditor0",
            "Effect": "Allow",
            "Action": "states:StartExecution",
            "Resource": "*"
        }
    ]
}

print(eventbridge_sfn_policy)

# Create Policy Object

In [ ]:
try:
    policy_eventbridge_sfn = iam.create_policy(
      PolicyName='DSOAWS_EventBridgeInvokeStepFunction',
      PolicyDocument=json.dumps(eventbridge_sfn_policy)
    )
    print("Done.")
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy already exists")
        policy_eventbridge_sfn_arn = f'arn:aws:iam::{account_id}:policy/DSOAWS_EventBridgeInvokeStepFunction'
        iam.create_policy_version(
            PolicyArn=policy_eventbridge_sfn_arn,
            PolicyDocument=json.dumps(eventbridge_sfn_policy),
            SetAsDefault=True)
        print("Policy updated.")
    else:
        print("Unexpected error: %s" % e)

# Get ARN

In [ ]:
policy_eventbridge_sfn_arn = f'arn:aws:iam::{account_id}:policy/DSOAWS_EventBridgeInvokeStepFunction'
print(policy_eventbridge_sfn_arn)

# Attach Policy To Role

In [ ]:
try:
    response = iam.attach_role_policy(
        PolicyArn=policy_eventbridge_sfn_arn,
        RoleName=iam_role_name_eventbridge
    )
    print("Done.")
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy is already attached. This is ok.")
    else:
        print("Unexpected error: %s" % e)

# Setup EventBridge Rule Target

In [ ]:
sfn = boto3.client('stepfunctions')

# Define Model Pipeline Inputs

In [ ]:
import time
timestamp = int(time.time())
print(timestamp)

In [ ]:
execution_name = 'run-{}'.format(timestamp)
print(execution_name)

# Retrieve Input Data S3 Locations

In [ ]:
%store -r processed_train_data_s3_uri
%store -r processed_validation_data_s3_uri
%store -r processed_test_data_s3_uri

# We are using the created Training Pipeline from notebook 02 and start a new run
We do this because we need the trained model from this previous run.  We could also choose to depend on a previously-trained model from an earlier section (ie. 07_train/) or we could manually copy the source to an S3 bucket and invoke that way.  This is exactly what the client-side SageMaker Python SDK does for us when we use `sagemaker.estimator.TensorFlow.fit()`, etc.

In [ ]:
inputs = {
  "Training": {
    "AlgorithmSpecification": {
      "TrainingImage": "763104351884.dkr.ecr.{}.amazonaws.com/tensorflow-training:2.1.0-cpu-py3".format(region),
      "TrainingInputMode": "File"
    },
    "OutputDataConfig": {
      "S3OutputPath": "s3://{}/training-pipeline-{}/models".format(bucket, execution_name)
    },
    "StoppingCondition": {
      "MaxRuntimeInSeconds": 7200
    },
    "ResourceConfig": {
      "InstanceCount": 1,
      "InstanceType": "ml.c5.2xlarge",
      "VolumeSizeInGB": 1024
    },
    "RoleArn": "{}".format(role),
    "InputDataConfig": [
      {
        "DataSource": {
          "S3DataSource": {
            "S3DataType": "S3Prefix",
            "S3Uri": "{}".format(processed_train_data_s3_uri),
            "S3DataDistributionType": "ShardedByS3Key"
          }
        },
        "ChannelName": "train"
      },
      {
        "DataSource": {
          "S3DataSource": {
            "S3DataType": "S3Prefix",
            "S3Uri": "{}".format(processed_validation_data_s3_uri),
            "S3DataDistributionType": "ShardedByS3Key"
          }
        },
        "ChannelName": "validation"
      },
      {
        "DataSource": {
          "S3DataSource": {
            "S3DataType": "S3Prefix",
            "S3Uri": "{}".format(processed_test_data_s3_uri),
            "S3DataDistributionType": "ShardedByS3Key"
          }
        },
        "ChannelName": "test"
      }
    ],
    "HyperParameters": {
      "epochs": "1",
      "learning_rate": "1e-05",
      "epsilon": "1e-08",
      "train_batch_size": "128",
      "validation_batch_size": "128",
      "test_batch_size": "128",
      "train_steps_per_epoch": "50",
      "validation_steps": "50",
      "test_steps": "50",
      "use_xla": "true",
      "use_amp": "true",
      "max_seq_length": "128",
      "freeze_bert_layer": "true",
      "enable_sagemaker_debugger": "false",
      "enable_checkpointing": "false",
      "enable_tensorboard": "false",        
      "run_validation": "true",
      "run_test": "true",
      "run_sample_predictions": "true",
      "sagemaker_submit_directory": "\"s3://{}/{}/estimator-source/source/sourcedir.tar.gz\"".format(bucket, stepfunction_name),
      "sagemaker_program": "\"tf_bert_reviews.py\"",
      "sagemaker_enable_cloudwatch_metrics": "false",
      "sagemaker_container_log_level": "20",
      "sagemaker_job_name": "\"training-pipeline-{}/estimator-source\"".format(execution_name),
      "sagemaker_region": "\"{}\"".format(region),
      "model_dir": "\"s3://{}/training-pipeline-{}/estimator-source/model\"".format(bucket, execution_name)
    },  
    "TrainingJobName": "estimator-training-pipeline-{}".format(execution_name),
    "DebugHookConfig": {
      "S3OutputPath": "s3://{}/".format(bucket)
    }
  },
  "Create Model": {
    "ModelName": "training-pipeline-{}".format(execution_name),
    "PrimaryContainer": {
      "Image": "763104351884.dkr.ecr.{}.amazonaws.com/tensorflow-inference:2.1-cpu".format(region),
      "Environment": {
        "SAGEMAKER_PROGRAM": "null",
        "SAGEMAKER_SUBMIT_DIRECTORY": "null",
        "SAGEMAKER_ENABLE_CLOUDWATCH_METRICS": "false",
        "SAGEMAKER_CONTAINER_LOG_LEVEL": "20",
        "SAGEMAKER_REGION": "{}".format(region)
      },
      "ModelDataUrl": "s3://{}/training-pipeline-{}/models/estimator-training-pipeline-{}/output/model.tar.gz".format(bucket, execution_name, execution_name)
    },
    "ExecutionRoleArn": "{}".format(role)
  },
  "Configure Endpoint": {
    "EndpointConfigName": "training-pipeline-{}".format(execution_name),
    "ProductionVariants": [
      {
        "InitialInstanceCount": 1,
        "InstanceType": "ml.m4.xlarge",
        "ModelName": "training-pipeline-{}".format(execution_name),
        "VariantName": "AllTraffic"
      }
    ]
  },
  "Deploy": {
    "EndpointConfigName": "training-pipeline-{}".format(execution_name),
    "EndpointName": "training-pipeline-{}".format(execution_name)
  }
}

In [ ]:
inputs_json = json.dumps(inputs)

print(inputs_json)

## Create EventBridge Rule Target

In [ ]:
# Check for exsting targets
targets = events.list_targets_by_rule(
    Rule='S3-Trigger',
    EventBusName='default'
)

In [ ]:
number_targets = len(targets['Targets'])

if number_targets > 0:
    for target in targets['Targets']:
        print(target['Id'])
        events.remove_targets(
            Rule='S3-Trigger',
            EventBusName='default',
            Ids=[target['Id']],
        Force=True
)
    print("Target: " +target['Id']+ " removed.")
else:
    print("No targets defined yet.")

In [ ]:
import uuid

target_id = str(uuid.uuid4())

response = events.put_targets(
    Rule='S3-Trigger',
    EventBusName='default',
    Targets=[
        {
            'Id': target_id,
            'Arn': stepfunction_arn,
            'RoleArn': iam_role_eventbridge_arn,
            'Input': inputs_json
        }
    ]
)

In [ ]:
print(response)

# Check Number of StepFunction Invocations **Before** the S3 Trigger

In [ ]:
response_before_uploading = sfn.list_executions(stateMachineArn=stepfunction_arn)
number_of_executions_before_uploading = len(response_before_uploading['executions'])
print(number_of_executions_before_uploading)

# Upload to S3 and Trigger a StepFunction Invocation

In [ ]:
!aws s3 cp  ./src/requirements.txt s3://$watched_bucket

# Check Number of StepFunction Invocations **After** the S3 Trigger

In [ ]:
import time
time.sleep(5)

In [ ]:
response_after_uploading = sfn.list_executions(stateMachineArn=stepfunction_arn)
number_of_executions_after_uploading = len(response_after_uploading['executions'])
print(number_of_executions_after_uploading)